In [1]:
%aiida

Loaded AiiDA DB environment - profile name: bmourino.

In [2]:
import re
import os
import pickle
import numpy as np
import pandas as pd
from glob import glob

from aiida import load_profile
from aiida.tools import get_kpoints_path
from aiida.orm.querybuilder import QueryBuilder
from aiida.plugins import CalculationFactory, WorkflowFactory
from aiida.orm import StructureData, Dict, load_node, CalcFunctionNode, CalcJobNode, WorkChainNode, BandsData, CifData

#import matplotlib.pyplot as plt

import sumo
from sumo.plotting.bs_plotter import SBSPlotter
from sumo.plotting.dos_plotter import SDOSPlotter
from sumo.electronic_structure.effective_mass import get_fitting_data, fit_effective_mass

from pymatgen.electronic_structure.core import Spin, Orbital
from pymatgen.electronic_structure.dos import Dos, CompleteDos
from pymatgen.electronic_structure.plotter import BSPlotter, DosPlotter, BSDOSPlotter
from pymatgen.electronic_structure.bandstructure import BandStructure, Kpoint, BandStructureSymmLine

HARTREE2EV = 27.211399

/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/pymatgen/electronic_structure/boltztrap.py:60: FutureWarning: which is deprecated; use which in shutil instead.
shutil.which has been available since Python 3.3. This will be removed in v2023.
  which("x_trans"),


In [3]:
load_profile()

In [4]:
def get_orbital_indices(orbitals):
    s_indices = []
    p_indices = []
    d_indices = []
    f_indices = []
    orbital_objects = []
    for i, orbital in enumerate(orbitals):
        if "s" in orbital: 
            s_indices.append(i)
        elif "p" in orbital: 
            p_indices.append(i)
        elif "d" in orbital: 
            d_indices.append(i)
        elif "f" in orbital: 
            f_indices.append(i)
        orbital_objects.append(Orbital(i))
    return s_indices, p_indices, d_indices, f_indices, orbital_objects


def get_bands_efermi(aiidaout_opt):
    with open(aiidaout_opt, 'r') as fh:
        for line in fh.readlines():
            if 'Fermi Energy' in line:
                bands_fermi = float(line.split()[-1])
                #bands_fermi = float(-18)
                print(f"bands fermi energy {bands_fermi}")
                return bands_fermi

def get_homo(aiidaout):
    with open(aiidaout, 'r') as fh:
        for line in fh.readlines():
            if 'Number of occupied orbitals' in line:
                homo = int(line.split()[-1])
                print(f"number of occupied orbitals {homo}")
                return homo

def read_from_pdos(fname, element_fractions, gaussian=0.01):

    HEADER_MATCH = re.compile(
    r'\# Projected DOS for atomic kind (?P<element>\w+) at iteration step i = \d+, E\(Fermi\) = [ \t]* (?P<Efermi>[^\t ]+) a\.u\.')

    # Column indexes, starting from 0
    EIGENVALUE_COLUMN = 1
    DENSITY_COLUMN = 3
    RES = 0.001
    
    with open(fname, 'r') as fh:
        match = HEADER_MATCH.match(fh.readline().rstrip())
     
        species = match.group('element')
        dos_fermi = float(match.group('Efermi')) * HARTREE2EV
        print('dos_fermi', dos_fermi)

        
        # header is originally: ['#', 'MO', 'Eigenvalue', '[a.u.]', 'Occupation', 's', 'py', ...]
        header = fh.readline().rstrip().split()[1:]  # remove the comment directly
        header[1:3] = [' '.join(header[1:3])]  # rejoin "Eigenvalue" and its unit
        data = np.loadtxt(fh)  # load the rest directly with numpy

        
    alldata = [data]
      
    s_indices, p_indices, d_indices, f_indices, _ = get_orbital_indices(header[DENSITY_COLUMN:])
    margin = 10 * gaussian
    emin = min(np.min(data[:, EIGENVALUE_COLUMN]) for data in alldata) - margin
    emax = max(np.max(data[:, EIGENVALUE_COLUMN]) for data in alldata) + margin
    ncols = sum(data.shape[1] - DENSITY_COLUMN for data in alldata)
    nmesh = int((emax-emin)/RES)+1  # calculate manually instead of using np.arange to ensure emax inside the mesh
    xmesh = np.linspace(emin, emax, nmesh)
    ymesh = np.zeros((nmesh, ncols))
    
    fact = RES/(gaussian*np.sqrt(2.0*np.pi))

    coloffset = 0
    for data in  alldata:
        ncol = data.shape[1] - DENSITY_COLUMN
        for idx in range(nmesh):
            func = np.exp(-(xmesh[idx]-data[:, EIGENVALUE_COLUMN])**2/(2.0*gaussian**2))*fact
            ymesh[idx, coloffset:(coloffset+ncol)] = func.dot(data[:, DENSITY_COLUMN:])

        coloffset += ncol
        
    xmesh *= HARTREE2EV

    pdoss = {}
    
    if s_indices: 
        pdoss['s'] = Dos(dos_fermi, xmesh, {Spin.up: np.sum(ymesh[:,s_indices], axis=1)})
    if p_indices: 
        pdoss['p'] = Dos(dos_fermi, xmesh, {Spin.up: np.sum(ymesh[:,p_indices], axis=1)})
    if d_indices: 
        pdoss['d'] = Dos(dos_fermi, xmesh, {Spin.up: np.sum(ymesh[:,d_indices], axis=1)})
    if f_indices: 
        pdoss['f'] = Dos(dos_fermi, xmesh, {Spin.up: np.sum(ymesh[:,f_indices], axis=1)})
    

    total_dos = Dos(dos_fermi, xmesh, {Spin.up: np.sum(ymesh, axis=1) } )
    
    
            
    return total_dos, {species: pdoss}


def get_dos_in_folder(folderpath, element_fractions={'O': 80, 'C': 192, 'H': 112, 'In': 16},
                      gaussian=0.01): 
    pdos_files = glob(os.path.join(folderpath, "*.pdos"))
    for pdos_file in pdos_files:
        if 'ALPHA' in pdos_file:
            raise NotImplementedError()
            
    pdoss = {}
    totals = []
    element_fractions = dict(zip(element_fractions.keys(), np.array(list(element_fractions.values()))/sum(element_fractions.values())))
        

    for file in pdos_files: 
        total, splitted = read_from_pdos(file, element_fractions, gaussian)
        pdoss.update(splitted)
        totals.append(total)
        
    complete_total = totals[0]
    
    for totaldos in totals[1:]: 
        complete_total += totaldos
        
    return complete_total, pdoss


def get_cellopt_retrieved_folder(wc_pk): 
    
    wc = load_node(wc_pk)
    for des in wc.called_descendants:
        if des.label == 'CELL_OPT': 
            folder = folder = des.get_retrieved_node()
            folder_path = folder._repository._get_base_folder().abspath
    
            return folder_path
    
    return None

# def get_bs_retrieved_folder(wc_pk): 
    
#     wc = load_node(wc_pk)
#     for des in wc.called_descendants:
#         if des.label == 'ENERGY': 
#             folder = folder = des.get_retrieved_node()
#             folder_path = folder._repository._get_base_folder().abspath
    
#             return folder_path
    
#     return None


def make_labels_dict(bandsdata):
    kps = bandsdata.get_kpoints()
    labels = bandsdata.labels
    
    label_dict = {}
    a = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    b = ['0', '1', '2', '3']
    
    for index, label in labels:
        if label == 'GAMMA':
            label = '\Gamma'
        if label == 'SIGMA':
            label = '\Sigma'            
        if label == 'DELTA':
            label = '\Delta' 
        if label == 'LAMBDA':
            label = '\Lambda'
        if label == 'GAMMA_0':
            label = '\Gamma'+'0'
        if label == 'SIGMA_0':
            label = '\Sigma'+'0'
        if label == 'DELTA_0':
            label = '\Delta'+'0'     
        if label == 'LAMBDA_0':
            label = '\Lambda'+'0'  
        if label == 'GAMMA_1':
            label = '\Gamma'+'1'
        if label == 'SIGMA_1':
            label = '\Sigma'+'1'
        if label == 'DELTA_1':
            label = '\Delta'+'1'     
        if label == 'LAMBDA_1':
            label = '\Lambda'+'1'  
        if label == 'GAMMA_2':
            label = '\Gamma'+'2'
        if label == 'SIGMA_2':
            label = '\Sigma'+'2' 
        if label == 'DELTA_2':
            label = '\Delta'+'2'     
        if label == 'LAMBDA_2':
            label = '\Lambda'+'2'
        for i in a:
            for j in b:
                if label == str(i)+'_'+str(j):
                    label = str(i)+str(j)
        label_dict[label] = kps[index]
        
    return label_dict


def run_seekpath(structure):
    parameters = Dict(dict={
            'symprec': 1e-06,
            'angle_tolerance': -1.0
        })
    seekpath_result = get_kpoints_path(structure, **parameters.get_dict())
    return seekpath_result['primitive_structure']



def call_sumo(bs, carrier='hole'):
    """
    Arguments:
        bs: Pymatgen BandStructureSymmLine object
        carrier
    Returns:
        (float) mass of lightest band of that type
    """
    masses = []
    if carrier == 'hole': 
        extreme = bs.get_vbm()
    elif carrier == 'electron': 
        extreme = bs.get_cbm()
        
    for spin in [Spin.up]:
        if carrier == 'electron':
            b_ind = list(extreme['band_index'][spin])
            b_ind_ext = min(b_ind)
        elif carrier == 'hole':
            b_ind = list(extreme['band_index'][spin])
            b_ind_ext = max(b_ind)

        for index in extreme['kpoint_index']:
            fit_data = get_fitting_data(bs, spin, 
                b_ind_ext, index, 4)[0]
            masses.append(fit_effective_mass(fit_data['distances'],
                fit_data['energies'], parabolic=False)) #try with parabolic=False
        
    return min([abs(mass) for mass in masses])


def get_pmg_bands(bands_data, pymatgen_structure, fermi):

    bs_pmg = BandStructureSymmLine(kpoints=bands_data.get_kpoints(), 
                                   eigenvals={Spin.up: bands_data.get_bands().T},
                                    lattice=pymatgen_structure.lattice.reciprocal_lattice, 
                                   efermi=fermi, 
                                   labels_dict=make_labels_dict(bands_data),
                                   structure=pymatgen_structure)
    
    return bs_pmg



def dump_as_pickle(obj, name): 
    with open(name, 'wb') as fh: 
        pickle.dump(obj, fh)


def plot_bandstructure(bs_pmg, name, dos_plotter, outdir='../output_data'):
    bsplotter = SBSPlotter(bs_pmg)
    plt = bsplotter.get_plot(dos_plotter=dos_plotter, dos_options={'plot_total': True}, dos_aspect=3)
    dump_as_pickle(dos_plotter, os.path.join(outdir, '_'.join([name, 'dosplotter'])+'.pkl'))
    dump_as_pickle(bs_pmg, os.path.join(outdir, '_'.join([name, 'bs_pmg'])+'.pkl'))
    plt.savefig(os.path.join(outdir, '{}_bands.pdf'.format(name)), bbox_inches='tight')
    #plt.title(name)
    plt.close('all')


def plot_dos(total_dos, ppdos, name, outdir='../output_data'):
    sumo.plotting.colour_cache.clear()
    dos_plotter = SDOSPlotter(total_dos, ppdos)
    #UNCOMMENT THE NEXT THREE LINES TO GET DOS PLOT
    plt = dos_plotter.get_plot(plot_total= False)
    plt.savefig(os.path.join(outdir, '{}_dos.pdf'.format(name)), bbox_inches='tight')
    plt.close('all')
    return dos_plotter


#def get_fermi(structure_label, df_femi=df):
#    f1 = df[df['label'] == structure_label]['fermi_spin_1_eV'].values[0]
#    f2 = df[df['label'] == structure_label]['fermi_spin_2_eV'].values[0]
    
#    return f1, f2


def get_bs_feats(bs_pmg):
    
    direct_band_gap = bs_pmg.get_direct_band_gap()
    #print('1', direct_band_gap)
    band_gap_dict = bs_pmg.get_band_gap()
    #print('2', band_gap_dict)
    direct = band_gap_dict['direct']
    #print('3', direct)
    gap_energy = band_gap_dict['energy']
    #print('4', gap_energy)
    transition = band_gap_dict['transition']
    vbm = bs_pmg.get_vbm()['energy']
    cbm = bs_pmg.get_cbm()['energy']
    print(f'vbm: {vbm}, cbm: {cbm}')
    effmass_hole = call_sumo(bs_pmg, 'hole')
    effmass_electron = call_sumo(bs_pmg, 'electron')
    
    return {
        'direct_band_gap_energy': direct_band_gap, 
        'direct_band_gap': direct,
        'band_gap_energy': gap_energy,
        'band_gap_transition': transition, 
        'vbm': vbm, 
        'cbm': cbm,
        'effective_mass_hole': effmass_hole,
        'effective_mass_electron': effmass_electron
    }

def analyze_bands_result(structure_label, structure_data, bands_data, fermi, dos_plotter): 
    structure_data_primitive = run_seekpath(structure_data)
    s_pmg = structure_data_primitive.get_pymatgen_structure()
    #print(s_pmg)
    results = {}
    pmg_bands = get_pmg_bands(bands_data, s_pmg, fermi)
    #print('1',bands_data)
    #print('2', fermi)
    print('3',pmg_bands.get_band_gap())
    #dump_as_pickle(pmg_bands, '../output_data'+structure_label+'_pmg'+'.pkl')
    results = get_bs_feats(pmg_bands)
    print('results')

    #UNCOMMENT THE NEXT 5 LINES TO GET BS PLOT
    print('plotting bs')
    try: 
        plot_bandstructure(pmg_bands, structure_label, dos_plotter)
    except Exception as e:
        print(e)
        pass
        
    return results

def analyze_from_qb_result(qb_result): 
    try:
        label = qb_result[0]
        cellopt_retrieved_folder = get_cellopt_retrieved_folder(qb_result[1])
        s = load_node(qb_result[-3])

        wc = load_node(qb_result[-2])
#         print(wc.called_descendants[2])
#         aiidaout_bands_folder = wc.called_descendants[2].get_retrieved_node().\
#         _repository._get_base_folder().abspath + "/"
#         aiida_out_bands_folder = get_bs_retrieved_folder(wc)
        aiidaout_bands_folder = wc.called_descendants[-1].get_retrieved_node().\
         _repository._get_base_folder().abspath + "/"    
    
        aiidaout_bands = aiidaout_bands_folder + "aiida.out"
        
        homo_bands = get_homo(aiidaout_bands)-1
        lumo_bands = homo_bands+1
        print(homo_bands,lumo_bands)
        
        aiidaout_opt = cellopt_retrieved_folder + "/aiida.out"
        
        print('getting dos')
    
        try:
            total_pdos, ppdos = get_dos_in_folder(cellopt_retrieved_folder,
                                              s.get_pymatgen().composition.as_dict(), 0.005)
            dos_plotter = plot_dos(total_pdos, ppdos, label)
        except Exception as e:
            dos_plotter = None
            print(e)
            pass
        
        print('getting bands')
        bands_data = load_node(qb_result[-1])

        
        print('analyzing bands')
        bands_efermi = get_bands_efermi(aiidaout_opt)
        ##add here
        vbm = max(bands_data.get_bands()[:,homo_bands])
        cbm = min(bands_data.get_bands()[:,lumo_bands])
        print(vbm,cbm)
        #print(aiidaout_opt)
        #print(bands_efermi)
        print(bands_data)
        result = analyze_bands_result(label, s, bands_data, vbm+0.00000001, dos_plotter)
        print(f"fermi getting returned: {total_pdos.efermi}")
        print(make_labels_dict(bands_data))
        result['label'] = label
        result['cellopt_retrieved'] = cellopt_retrieved_folder

        return result
    except Exception as e:
        print(e)
        return None

In [5]:
Cp2kCalculation = CalculationFactory('cp2k')
Cp2kBaseWorkChain = WorkflowFactory('cp2k.base')
Cp2kMultistageWorkChain = WorkflowFactory('lsmo.cp2k_multistage')
Cp2kBandsWorkChain = WorkflowFactory('photocat_workchains.bandstructure')
Cp2kPhotoCatWorkChain = WorkflowFactory('photocat_workchains.cp2k_photocat')
Cp2kOPtPlusUWorkChain = WorkflowFactory('photocat_workchains.cp2k_opt_plusu')

In [6]:
qb = QueryBuilder()
qb.append(Group, filters={'label': 'cofs-herows'}, tag='group')

In [7]:
qb.append(Cp2kPhotoCatWorkChain, with_group='group', tag='photocat') #specifying the workchain I want to inspect; if I don't put the uuid it won't be in the dict
qb.append(CalcFunctionNode, filters={'label': {'==': 'get_structure_from_cif'}}, tag='get_st', with_incoming='photocat') #this is to be able to append CifData (it's important because their labels are the structure name)
qb.append(CifData, project=['label'], with_outgoing='get_st', tag='structure_label') #appending CifData to later get the structure name
qb.append(Cp2kMultistageWorkChain, with_incoming='photocat', filters={'and': [{'attributes.process_state':{'==':'finished'}}]}, project='uuid', tag='ms_wc') #appending multistage (all of them, including the single point)
qb.append(Cp2kBaseWorkChain, with_incoming='ms_wc', filters={'label': {'==':'stage_1_settings_0'}}, project='uuid', tag='cellopt') #appending multistage (all of them, including the single point)
qb.append(StructureData, project=['uuid'], with_outgoing='cellopt') #structures used for ms_wcqb.append(Dict, with_incoming='ms_wc', project=['uuid'], filters={'attributes': {'has_key': 'natoms'}}) #still don't know what this is for
qb.append(StructureData, project=['uuid'], with_outgoing='ms_wc') #structures used for ms_wc
# qb.append(Dict, with_incoming='ms_wc', project=['uuid'], filters={'attributes': {'has_key': 'natoms'}}) #still don't know what this is for
qb.append(StructureData, with_incoming='ms_wc', project=['uuid'], tag='relaxed') #relaxed structure; will filter ms_wc that are not single point (not appending anymore the single point ones, since the structures here have to be generated by a ms_wc - incoming - and a single point ms_wc won't generate structures)
qb.append(CalcFunctionNode, filters={'label': {'==': 'structure_with_pbc'}}, project=['uuid'], with_incoming='photocat', tag='get_pbc_calcfunction') #this is to be able to append primitive structures
qb.append(StructureData, with_incoming='get_pbc_calcfunction', project=['uuid'], tag='structure_with_pbc') #appending primitive structures so I can use them for plotting bands
qb.append(Cp2kBandsWorkChain, tag='bands_wc', project=['uuid']) #this is apending the bandst workchain
# qb.append(Cp2kBandsWorkChain, with_incoming='relaxed', filters={'label': {'==': 'bs_prim_test'}}, tag='bands_wc', project=['uuid']) #this is apending the bandst workchain - 2 - uncomment here and comment the previous one for the bs after tests
#, filters={'label': {'==': 'bs2'}}, 
#qb.append(Cp2kBaseWorkChain, tag='cp2k_base', with_incoming='bands_wc') #this is apending the bandst workchain
#qb.append(Cp2kCalculation, tag='cp2k_calc', with_incoming='cp2k_base') #this is apending the bandst workchain
#qb.append(StructureData, project=['uuid'], with_outgoing='cp2k_calc')
#qb.append(CalcFunctionNode, filters={'type': {'==': 'seekpath_structure_analysis'}}, tag='seekpath') #this is to be able to append primitive structures
#qb.append(StructureData, with_outgoing='seekpath', project=['uuid'], tag='primitive') #appending primitive structures so I can use them for plotting bands
#qb.append(Cp2kBaseWorkChain, with_incoming='bands_wc', tag='base_bands')
#qb.append(Cp2kCalculation, with_incoming='base_bands', tag='calc_bands')
#qb.append(StructureData, project=['uuid'], with_outgoing='calc_bands')
#maybe I should append the structure that is used for the bands wc? I'm getting different results - didn't work, so instead I'm doing seekpath analysis here
qb.append(BandsData, project=['uuid'], with_incoming='bands_wc') #this is appending the data from bands_wc, i.e., kpoints, bands, labels

/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/aiida/orm/querybuilder.py:271: AiidaEntryPointWarning: Process type 'aiida_lsmo.workchains.cp2k_multistage.Cp2kMultistageWorkChain' does not correspond to a registered entry. This risks queries to fail once the location of the process class changes. Add an entry point for 'aiida_lsmo.workchains.cp2k_multistage.Cp2kMultistageWorkChain' to remove this warning.
  "Add an entry point for '{value}' to remove this warning.".format(value=value), AiidaEntryPointWarning
/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/aiida/orm/querybuilder.py:271: AiidaEntryPointWarning: Process type 'aiida_cp2k.workchains.base.Cp2kBaseWorkChain' does not correspond to a registered entry. This risks queries to fail once the location of the process class changes. Add an entry point for 'aiida_cp2k.workchains.base.Cp2kBaseWorkChain' to remove this warning.
  "Add an entry point for '{value}' to remove this warning.".format(value

In [8]:
# qb.append(Cp2kOPtPlusUWorkChain, with_group='group', tag='photocat') #specifying the workchain I want to inspect; if I don't put the uuid it won't be in the dict
# qb.append(CalcFunctionNode, filters={'label': {'==': 'get_structure_from_cif'}}, tag='get_st', with_incoming='photocat') #this is to be able to append CifData (it's important because their labels are the structure name)
# qb.append(CifData, project=['label'], with_outgoing='get_st', tag='structure_label') #appending CifData to later get the structure name
# qb.append(Cp2kMultistageWorkChain, with_incoming='photocat', filters={'and': [{'attributes.process_state':{'==':'finished'}}]}, project='uuid', tag='ms_wc') #appending multistage (all of them, including the single point)
# qb.append(StructureData, project=['uuid'], with_outgoing='ms_wc') #structures used for ms_wc
# qb.append(Dict, with_incoming='ms_wc', project=['uuid'], filters={'attributes': {'has_key': 'natoms'}}) #still don't know what this is for
# qb.append(StructureData, with_incoming='ms_wc', project=['uuid'], tag='relaxed') #relaxed structure; will filter ms_wc that are not single point (not appending anymore the single point ones, since the structures here have to be generated by a ms_wc - incoming - and a single point ms_wc won't generate structures)

In [9]:
len(qb.all())

10

In [10]:
qb_results_dict = {}
qb_results  = []
names = []
## eliminate duplicates:
for result in qb.all():

    qb_results_dict[result[0]] = result
    if result[0] in names:
        continue
    else:
        names.append(result[0])
        qb_results.append(result)
len(qb_results)

5

In [11]:
qb_results

[['ATEBPY-AB-1Cells.cif',
  '7ab83677-4a0b-46f5-a31f-e658519dc453',
  '71b27655-03fe-45a3-ae5d-ced21d60b440',
  'f839c4b5-5728-4fe1-83cd-e6efcfab367c',
  'b9dfd8e8-04d8-4d97-8fb0-4361e0c20327',
  '7c43755d-a1a8-49b5-a962-37fb278aee68',
  'f8e02855-eb47-426e-8c44-9dc93d22f1bb',
  '3718693a-71d1-4ce3-8cb6-d3af52e6b949',
  '4b946595-551e-48ca-ae18-761c6330cbbc',
  'a548ac66-c0e6-45ad-98dd-8bfc8dc140f4'],
 ['ATEPPy-AA-p2m.cif',
  'f6a6b384-4f85-4441-ad1b-6fe2a9e7107d',
  'ab06b5b9-c11c-4b6e-b14e-a928cff43350',
  'fd1f91b9-f523-4b77-9524-4c0e69e92f33',
  'd2d5ea1e-acff-47be-ad81-520febb7166b',
  'ec46fcf0-d8d6-4863-89b6-c38665a06792',
  '1bddfa26-c5ba-4b1a-9d5f-9aba9a3571a1',
  'f5aab497-0e4a-4430-a110-0b62fc83b8e5',
  'e73fdf86-e06f-4841-86f9-fa7b3b972ad2',
  '780fd382-5c04-474f-9901-e1f4ec5d89dd'],
 ['ATEPPY-AAsl-p21m.cif',
  '4cf514c5-f10f-4886-ba94-832e7218a189',
  '5dfff510-5c96-4bd0-bab2-6a85a5e59b72',
  '0e75a531-12a3-4feb-98ab-fd1497a31732',
  '84de6076-ed8e-4f46-a5be-d8cfe39ba2d7',

In [12]:
label = []
for i,j in enumerate(qb_results[0:]):
    label.append(j[0])
print(label)

['ATEBPY-AB-1Cells.cif', 'ATEPPy-AA-p2m.cif', 'ATEPPY-AAsl-p21m.cif', 'ATENPY-AAsl-p21m.cif', 'ATEBPY-AAsl-p21m.cif']


In [13]:
all_results = []
for i, qb_result in enumerate(qb_results[0:]):
    #print(qb_result)
    print(f"{i+1}/{len(qb_results[0:])}")
    for node in qb_result:
        try:
            print(load_node(node).get_dict()['warnings'])
        except:
            pass
    
    print(qb_result[0])
    try:
        r = analyze_from_qb_result(qb_result)
        all_results.append(r)
    except Exception as e:
        print(e)
        pass

    
all_results_filtred = [r  for r in all_results if r is not None]
df_results = pd.DataFrame(all_results_filtred)
df_results.to_csv('../output_data/bands_data.csv', index=False)
print(df_results)

1/5
ATEBPY-AB-1Cells.cif
number of occupied orbitals 127
126 127
getting dos
dos_fermi -1.699106964959
dos_fermi -1.699106964959
dos_fermi -1.699106964959
getting bands
analyzing bands
bands fermi energy -1.699095
-1.70078746 -0.75161152
uuid: a548ac66-c0e6-45ad-98dd-8bfc8dc140f4 (pk: 14436)


/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:314: EdgeCaseWarning: aP lattice, but the k_beta3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)
/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:318: EdgeCaseWarning: aP lattice, but the k_gamma3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)


3 {'direct': True, 'energy': 0.9491759399999999, 'transition': 'Y2-(-0.488,0.512,0.000)'}
vbm: -1.70078746, cbm: -0.75161152
results
plotting bs
fermi getting returned: -1.699106964959
{'\\Gamma': array([0., 0., 0.]), 'Y2': array([-0.5,  0.5,  0. ]), 'M2': array([-0.5,  0.5,  0.5]), 'A': array([0. , 0. , 0.5]), 'L2': array([0. , 0.5, 0.5]), 'V2': array([0. , 0.5, 0. ])}
2/5
ATEPPy-AA-p2m.cif
number of occupied orbitals 127
126 127
getting dos
dos_fermi -2.333676789639
dos_fermi -2.333676789639
dos_fermi -2.333676789639
getting bands
analyzing bands
bands fermi energy -2.333669
-2.33365927 -2.15443443
uuid: 780fd382-5c04-474f-9901-e1f4ec5d89dd (pk: 14458)


/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:314: EdgeCaseWarning: aP lattice, but the k_beta3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)
/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:318: EdgeCaseWarning: aP lattice, but the k_gamma3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)


3 {'direct': False, 'energy': 0.17922484000000027, 'transition': '\\Gamma-Z'}
vbm: -2.33365927, cbm: -2.15443443
results
plotting bs
fermi getting returned: -2.333676789639
{'\\Gamma': array([0., 0., 0.]), 'Z': array([0. , 0.5, 0. ]), 'D': array([0. , 0.5, 0.5]), 'B': array([0. , 0. , 0.5]), 'A': array([-0.5,  0. ,  0.5]), 'E': array([-0.5,  0.5,  0.5]), 'C2': array([-0.5,  0.5,  0. ]), 'Y2': array([-0.5,  0. ,  0. ])}
3/5
ATEPPY-AAsl-p21m.cif
number of occupied orbitals 254
253 254
getting dos
dos_fermi -2.398195016668
dos_fermi -2.398195016668
dos_fermi -2.398195016668
getting bands
analyzing bands
bands fermi energy -2.398197
-2.39804299 -1.66707204
uuid: f781b8d8-059c-4d95-9fb6-70c99be0fc71 (pk: 14740)


...

3 {'direct': True, 'energy': 0.7309709499999999, 'transition': '\\Gamma-\\Gamma'}
vbm: -2.39804299, cbm: -1.66707204
results
plotting bs
fermi getting returned: -2.398195016668
{'\\Gamma': array([0., 0., 0.]), 'Z': array([0. , 0.5, 0. ]), 'D': array([0. , 0.5, 0.5]), 'B': array([0. , 0. , 0.5]), 'A': array([-0.5,  0. ,  0.5]), 'E': array([-0.5,  0.5,  0.5]), 'C2': array([-0.5,  0.5,  0. ]), 'Y2': array([-0.5,  0. ,  0. ])}
4/5
ATENPY-AAsl-p21m.cif
number of occupied orbitals 254
253 254
getting dos
dos_fermi -2.08548161936
dos_fermi -2.08548161936
dos_fermi -2.08548161936
getting bands
analyzing bands
bands fermi energy -2.085481
-2.08642261 -1.30995732
uuid: b42a1f84-3e77-4a62-99a3-090d7f2a0ea8 (pk: 14684)


/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:314: EdgeCaseWarning: aP lattice, but the k_beta3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)
/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:318: EdgeCaseWarning: aP lattice, but the k_gamma3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)


3 {'direct': True, 'energy': 0.77646529, 'transition': '(0.000,0.000,0.100)-(0.000,0.000,0.100)'}
vbm: -2.08642261, cbm: -1.30995732
results
plotting bs
fermi getting returned: -2.08548161936
{'\\Gamma': array([0., 0., 0.]), 'Z': array([0. , 0.5, 0. ]), 'D': array([0. , 0.5, 0.5]), 'B': array([0. , 0. , 0.5]), 'A': array([-0.5,  0. ,  0.5]), 'E': array([-0.5,  0.5,  0.5]), 'C2': array([-0.5,  0.5,  0. ]), 'Y2': array([-0.5,  0. ,  0. ])}
5/5
ATEBPY-AAsl-p21m.cif
number of occupied orbitals 254
253 254
getting dos
dos_fermi -1.871436754826
dos_fermi -1.871436754826
dos_fermi -1.871436754826
getting bands
analyzing bands
bands fermi energy -1.871424
-1.86082085 -1.04495144
uuid: 69fd8a5a-e10d-4f8f-b303-24449cbcd8c4 (pk: 14679)


/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:310: EdgeCaseWarning: aP lattice, but the k_alpha3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)
/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/seekpath/hpkot/__init__.py:318: EdgeCaseWarning: aP lattice, but the k_gamma3 angle is almost equal to 90 degrees
  "to 90 degrees", EdgeCaseWarning)


3 {'direct': False, 'energy': 0.8158694100000001, 'transition': '(0.000,0.000,0.225)-(0.000,0.175,0.000)'}
vbm: -1.86082085, cbm: -1.04495144
results
plotting bs
fermi getting returned: -1.871436754826
{'\\Gamma': array([0., 0., 0.]), 'Z': array([0. , 0.5, 0. ]), 'D': array([0. , 0.5, 0.5]), 'B': array([0. , 0. , 0.5]), 'A': array([-0.5,  0. ,  0.5]), 'E': array([-0.5,  0.5,  0.5]), 'C2': array([-0.5,  0.5,  0. ]), 'Y2': array([-0.5,  0. ,  0. ])}
   direct_band_gap_energy  direct_band_gap  band_gap_energy  \
0                0.949176             True         0.949176   
1                0.828939            False         0.179225   
2                0.730971             True         0.730971   
3                0.776465             True         0.776465   
4                0.821888            False         0.815869   

                       band_gap_transition       vbm       cbm  \
0                  Y2-(-0.488,0.512,0.000) -1.700787 -0.751612   
1                                 \Ga

In [570]:
# print(load_node('9a875c8b-8484-449f-81f9-94daa6b1bafd').pk) #-0.710325 eV fermi energy; 07001N2.cif
# print(load_node('6b5751b1-aece-4bd0-8d46-e5824d13847d').pk) #-1.571405; 05000N2.cif
# print(load_node('5133c2d1-6fd0-46e4-90ab-0d6e9cb06d50').pk) #-2.407659; 1.cif
# ##why don't those work with pymatgen for plotting band structure????